# Example of Training with MotionSense

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pandas as pd
from pathlib import Path

sys.path.append("..")

## Read Processed Data

In [3]:
from librep.utils.dataset import PandasDatasetsIO

In [4]:
dataset_path = Path("../data/views/MotionSense/balanced_view")
train, validation, test = PandasDatasetsIO(dataset_path).load()

In [5]:
train = pd.concat([train, validation], ignore_index=True)
train

,Unnamed: 0,attitude.roll-0,attitude.roll-1,attitude.roll-2,attitude.roll-3,attitude.roll-4,attitude.roll-5,attitude.roll-6,attitude.roll-7,attitude.roll-8,...,userAcceleration.z-295,userAcceleration.z-296,userAcceleration.z-297,userAcceleration.z-298,userAcceleration.z-299,activity code,length,trial_code,index,user
0,0,1.093387,1.129419,1.166286,1.176539,1.218638,1.218076,1.169095,1.104227,1.041327,...,0.025803,0.015770,0.109555,0.172113,0.248941,0,300,1,1200,16
1,1,1.528132,1.527992,1.527765,1.516768,1.493941,1.476302,1.455153,1.441702,1.443440,...,0.578416,0.572674,0.490945,0.345010,0.129649,0,300,1,0,1
2,2,-1.331274,-1.315688,-1.315034,-1.315621,-1.287623,-1.218232,-1.139884,-1.054611,-0.970854,...,0.102089,0.124263,0.122183,0.125418,0.230901,0,300,2,600,21
3,3,0.715184,0.679044,0.662668,0.662928,0.670537,0.666622,0.652839,0.662879,0.723967,...,-0.033105,0.119528,0.057709,-0.057865,-0.097547,0,300,11,0,7
4,4,-2.004835,-2.041697,-2.044302,-2.038474,-2.055756,-2.102689,-2.158746,-2.202283,-2.232511,...,0.072444,0.005214,-0.231522,-0.560348,-0.678777,0,300,2,1800,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1909,325,-0.809114,-0.504596,-0.200415,-0.069472,-0.009363,-0.019789,-0.084377,-0.162347,-0.184036,...,-0.754819,-0.570919,-1.006368,-1.232031,-0.321935,5,300,16,0,6
1910,326,1.225254,1.219554,1.262265,1.213586,1.229215,1.309294,1.371141,1.403459,1.491145,...,1.737598,1.105376,-0.184008,-1.426659,-1.711004,5,300,9,4500,2
1911,327,3.138105,-3.105622,-3.073557,-3.034694,-2.983038,-2.938967,-2.891639,-2.733605,-1.150408,...,-1.048692,-0.806513,-0.533924,1.383309,-0.669395,5,300,9,0,6
1912,328,1.476984,1.582842,1.692089,1.797948,1.852740,1.886674,1.885654,1.810233,1.708376,...,-0.326163,-0.657696,-0.787899,0.421130,0.055916,5,300,9,600,2


## Create the MotionSense Dataset

In [6]:
from librep.datasets.common import PandasMultiModalDataset

In [7]:
features = [
    "userAcceleration.x",
    "userAcceleration.y",
    "userAcceleration.z",
    "rotationRate.x",
    "rotationRate.y",
    "rotationRate.z"
]

train_dataset = PandasMultiModalDataset(
    train,
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)
validation_dataset = PandasMultiModalDataset(
    validation,
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)
test_dataset = PandasMultiModalDataset(
    test,
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)

In [8]:
train_dataset[0]

(array([-0.120843,  0.470622,  0.269861, ..., -0.046833, -0.283947,
        -0.521564]),
 0)

In [9]:
train_dataset.window_slices, train_dataset.window_names, train_dataset.num_windows

([(0, 300), (300, 600), (600, 900), (900, 1200), (1200, 1500), (1500, 1800)],
 ['userAcceleration.x',
  'userAcceleration.y',
  'userAcceleration.z',
  'rotationRate.x',
  'rotationRate.y',
  'rotationRate.z'],
 6)

In [10]:
from librep.datasets.common import TransformMultiModalDataset
from librep.transforms.fft import FFT

In [11]:
fft_transform = FFT()
transformer = TransformMultiModalDataset(transforms=[fft_transform], new_window_name_prefix="fft.")
train_dataset_fft = transformer(train_dataset)
validation_dataset_fft = transformer(validation_dataset)
test_dataset_fft = transformer(test_dataset)
train_dataset_fft.window_slices, train_dataset_fft.window_names, train_dataset_fft.num_windows

([(0, 300), (300, 600), (600, 900), (900, 1200), (1200, 1500), (1500, 1800)],
 ['fft.userAcceleration.x',
  'fft.userAcceleration.y',
  'fft.userAcceleration.z',
  'fft.rotationRate.x',
  'fft.rotationRate.y',
  'fft.rotationRate.z'],
 6)

In [12]:
from librep.datasets.common import TransformMultiModalDataset
from librep.transforms.fft import FFT
from librep.transforms.stats import StatsTransform

fft_transform = FFT()
stats_transform = StatsTransform()
transformer = TransformMultiModalDataset(transforms=[fft_transform, stats_transform], new_window_name_prefix="fft-stats.")
train_dataset_fft_stats = transformer(train_dataset)
validation_dataset_fft_stats = transformer(validation_dataset)
test_dataset_fft_stats = transformer(test_dataset)
train_dataset_fft_stats.window_slices, train_dataset_fft_stats.window_names, train_dataset_fft_stats.num_windows

([(0, 19), (19, 38), (38, 57), (57, 76), (76, 95), (95, 114)],
 ['fft-stats.userAcceleration.x',
  'fft-stats.userAcceleration.y',
  'fft-stats.userAcceleration.z',
  'fft-stats.rotationRate.x',
  'fft-stats.rotationRate.y',
  'fft-stats.rotationRate.z'],
 6)

In [13]:
from librep.datasets.common import combine_multi_modal_datasets
combined_train_dset = combine_multi_modal_datasets(train_dataset_fft, train_dataset_fft_stats)
combined_test_dset = combine_multi_modal_datasets(test_dataset_fft, test_dataset_fft_stats)
combined_train_dset.window_slices, combined_train_dset.window_names, combined_train_dset.num_windows

([(0, 300),
  (300, 600),
  (600, 900),
  (900, 1200),
  (1200, 1500),
  (1500, 1800),
  (1800, 1819),
  (1819, 1838),
  (1838, 1857),
  (1857, 1876),
  (1876, 1895),
  (1895, 1914)],
 ['fft.userAcceleration.x',
  'fft.userAcceleration.y',
  'fft.userAcceleration.z',
  'fft.rotationRate.x',
  'fft.rotationRate.y',
  'fft.rotationRate.z',
  'fft-stats.userAcceleration.x',
  'fft-stats.userAcceleration.y',
  'fft-stats.userAcceleration.z',
  'fft-stats.rotationRate.x',
  'fft-stats.rotationRate.y',
  'fft-stats.rotationRate.z'],
 12)

In [14]:
from librep.datasets.common import TransformMultiModalDataset
from librep.transforms.fft import FFT
from librep.transforms.resampler import SimpleResampler
from librep.transforms.stats import StatsTransform

resampler = SimpleResampler(new_sample_size=100)
fft_transform = FFT()
stats_transform = StatsTransform()
transformer = TransformMultiModalDataset(transforms=[resampler, fft_transform], new_window_name_prefix="resample-100-fft.")
resampled_train_dataset_fft_stats = transformer(train_dataset)
resampled_validation_dataset_fft_stats = transformer(validation_dataset)
resampled_test_dataset_fft_stats = transformer(test_dataset)
resampled_train_dataset_fft_stats.window_slices, resampled_train_dataset_fft_stats.window_names, resampled_train_dataset_fft_stats.num_windows

([(0, 100), (100, 200), (200, 300), (300, 400), (400, 500), (500, 600)],
 ['resample-100-fft.userAcceleration.x',
  'resample-100-fft.userAcceleration.y',
  'resample-100-fft.userAcceleration.z',
  'resample-100-fft.rotationRate.x',
  'resample-100-fft.rotationRate.y',
  'resample-100-fft.rotationRate.z'],
 6)

## Train

In [15]:
from librep.utils.workflow import SimpleTrainEvalWorkflow, MultiRunWorkflow
from librep.estimators import RandomForestClassifier
from librep.metrics.report import ClassificationReport
import yaml

reporter = ClassificationReport(use_accuracy=True, use_f1_score=True, use_classification_report=False, use_confusion_matrix=False, plot_confusion_matrix=False)
experiment = SimpleTrainEvalWorkflow(estimator=RandomForestClassifier, do_not_instantiate=False, do_fit=True, evaluator=reporter)
multi_run_experiment = MultiRunWorkflow(workflow=experiment, num_runs=3, debug=False)

In [16]:
result = multi_run_experiment(train_dataset, [test_dataset, validation_dataset, test_dataset])
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1661358495.6756208
    result:
    -   accuracy: 0.8170731707317073
        f1 score (macro): 0.8151771512943607
        f1 score (micro): 0.8170731707317073
        f1 score (weighted): 0.818969190169054
    -   accuracy: 1.0
        f1 score (macro): 1.0
        f1 score (micro): 1.0
        f1 score (weighted): 1.0
    -   accuracy: 0.8170731707317073
        f1 score (macro): 0.8151771512943607
        f1 score (micro): 0.8170731707317073
        f1 score (weighted): 0.818969190169054
    run id: 1
    start: 1661358492.0876389
    time taken: 3.587981939315796
-   end: 1661358499.126598
    result:
    -   accuracy: 0.8028455284552846
        f1 score (macro): 0.798936696907584
        f1 score (micro): 0.8028455284552846
        f1 score (weighted): 0.8067543600029852
    -   accuracy: 1.0
        f1 score (macro): 1.0
        f1 score (micro): 1.0
        f1 score (weighted): 1.0
    -   accuracy: 0.8028455284552846
        f1 score (macro): 0.798936696907584
    

In [17]:
result = multi_run_experiment(train_dataset_fft, test_dataset_fft)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1661358505.8749197
    result:
    -   accuracy: 0.8739837398373984
        f1 score (macro): 0.8749246057323559
        f1 score (micro): 0.8739837398373984
        f1 score (weighted): 0.8730428739424408
    run id: 1
    start: 1661358502.6077354
    time taken: 3.267184257507324
-   end: 1661358509.127251
    result:
    -   accuracy: 0.8739837398373984
        f1 score (macro): 0.8746529985547061
        f1 score (micro): 0.8739837398373984
        f1 score (weighted): 0.8733144811200905
    run id: 2
    start: 1661358505.8749216
    time taken: 3.2523293495178223
-   end: 1661358512.3787897
    result:
    -   accuracy: 0.8821138211382114
        f1 score (macro): 0.8822914007362329
        f1 score (micro): 0.8821138211382114
        f1 score (weighted): 0.88193624154019
    run id: 3
    start: 1661358509.1272526
    time taken: 3.2515370845794678



In [18]:
result = multi_run_experiment(train_dataset_fft_stats, test_dataset_fft_stats)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1661358513.014757
    result:
    -   accuracy: 0.8841463414634146
        f1 score (macro): 0.8834618755170088
        f1 score (micro): 0.8841463414634146
        f1 score (weighted): 0.8848308074098206
    run id: 1
    start: 1661358512.406693
    time taken: 0.6080639362335205
-   end: 1661358513.6241663
    result:
    -   accuracy: 0.8943089430894309
        f1 score (macro): 0.8938535908395183
        f1 score (micro): 0.8943089430894309
        f1 score (weighted): 0.8947642953393434
    run id: 2
    start: 1661358513.014759
    time taken: 0.6094071865081787
-   end: 1661358514.1949937
    result:
    -   accuracy: 0.8943089430894309
        f1 score (macro): 0.8937693982024348
        f1 score (micro): 0.8943089430894309
        f1 score (weighted): 0.8948484879764268
    run id: 3
    start: 1661358513.624168
    time taken: 0.5708258152008057



In [19]:
result = multi_run_experiment(combined_train_dset, combined_test_dset)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1661358517.3558223
    result:
    -   accuracy: 0.9146341463414634
        f1 score (macro): 0.9144495847390727
        f1 score (micro): 0.9146341463414634
        f1 score (weighted): 0.9148187079438542
    run id: 1
    start: 1661358514.2219176
    time taken: 3.1339046955108643
-   end: 1661358520.3571053
    result:
    -   accuracy: 0.9207317073170732
        f1 score (macro): 0.9206176698645715
        f1 score (micro): 0.9207317073170733
        f1 score (weighted): 0.9208457447695747
    run id: 2
    start: 1661358517.3558238
    time taken: 3.001281499862671
-   end: 1661358523.4246764
    result:
    -   accuracy: 0.9146341463414634
        f1 score (macro): 0.9145765580734723
        f1 score (micro): 0.9146341463414634
        f1 score (weighted): 0.9146917346094545
    run id: 3
    start: 1661358520.357107
    time taken: 3.0675694942474365



In [20]:
result = multi_run_experiment(resampled_train_dataset_fft_stats, resampled_test_dataset_fft_stats)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1661358525.2980206
    result:
    -   accuracy: 0.8922764227642277
        f1 score (macro): 0.8928631115978655
        f1 score (micro): 0.8922764227642277
        f1 score (weighted): 0.8916897339305897
    run id: 1
    start: 1661358523.4526699
    time taken: 1.845350742340088
-   end: 1661358527.0441542
    result:
    -   accuracy: 0.8943089430894309
        f1 score (macro): 0.8954620898421632
        f1 score (micro): 0.8943089430894309
        f1 score (weighted): 0.8931557963366987
    run id: 2
    start: 1661358525.2980227
    time taken: 1.746131420135498
-   end: 1661358528.8356023
    result:
    -   accuracy: 0.8882113821138211
        f1 score (macro): 0.8889239243704087
        f1 score (micro): 0.8882113821138212
        f1 score (weighted): 0.8874988398572334
    run id: 3
    start: 1661358527.0441558
    time taken: 1.7914464473724365

